<a href="https://colab.research.google.com/github/ednilsonmcs/analysis-pyspark/blob/main/TrabalhoPyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**##InstalandooJava**

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

**##Fazendo download**

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

**##Descompactando os arquivos**

In [3]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

**##Importando a biblioteca os**

In [4]:
import os

**##Definindo a variávelde ambiante JAVA**

In [5]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

**##Definindo a variávelde ambiente do SPARK**

In [6]:
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

**##Instalando o findspark**

In [7]:
!pip install -q findspark

**##Importando a findspark**

In [8]:
import findspark 

**##Iniciando o findspark**

In [9]:
findspark.init()

**##importando o pacote necessário para iniciar uma seção Spark**

In [10]:
from pyspark.sql import SparkSession

In [11]:
from pyspark.sql.functions import *

In [12]:
from pyspark.sql.types import IntegerType

In [13]:
from pyspark.sql.functions import  month, year, to_date, date_format

**##iniciando o sparkcontext**

In [14]:
sc = SparkSession.builder.master('local[*]').getOrCreate() 

**##Verificando se a sessão foi criada**

In [15]:
sc.version

'3.1.2'

In [16]:
dp = sc.read.options(header=True).csv(["/tmp/bombeiros"])

In [17]:
dp = dp.withColumn("data_us",to_date(dp.Data, "dd/MM/yyyy"))

In [18]:
dp = dp.withColumn("mes", month(dp.data_us))

In [19]:
dp = dp.withColumn("ano", year(dp.data_us))

In [20]:
dp = dp.withColumn("Valor Bruto (R$)", regexp_replace(dp['Valor Bruto (R$)'], "[^0-9.]", ""))

In [21]:
dp = dp.withColumn("Valor Bruto (R$)", regexp_replace(dp['Valor Bruto (R$)'], "\\.", ""))

In [22]:
dp = dp.withColumn("Valor Bruto (R$)", dp['Valor Bruto (R$)'].cast("float"))

In [23]:
dp.groupBy("mes", "ano").agg(sum("Valor Bruto (R$)").alias("valor_total")).orderBy("valor_total", ascending=False)

DataFrame[mes: int, ano: int, valor_total: double]

In [24]:
rank_mes_maior_gastos = dp.groupBy("mes", "ano").agg(sum("Valor Bruto (R$)").alias("valor_total")).orderBy("valor_total", ascending=False)

In [25]:
rank_mes_menor_gastos = dp.groupBy("mes", "ano").agg(sum("Valor Bruto (R$)").alias("valor_total")).orderBy("valor_total", ascending=True)

In [26]:
media_mensal_anual = dp.groupBy("ano").agg(avg("Valor Bruto (R$)").alias("media_mensal")).orderBy("ano", ascending=False)

In [27]:
rank_ano_maior_gastos = dp.groupBy("ano").agg(sum("Valor Bruto (R$)").alias("valor_total")).orderBy("valor_total", ascending=False)

In [28]:
rank_ano_menor_gastos = dp.groupBy("ano").agg(sum("Valor Bruto (R$)").alias("valor_total")).orderBy("valor_total", ascending=True)

In [ ]:
rank_mes_maior_gastos.show()

+---+----+-------------+
|mes| ano|  valor_total|
+---+----+-------------+
| 12|2017| 2.19374784E9|
| 10|2016| 1.26285459E9|
|  9|2021|1.089494821E9|
| 12|2019|1.064163513E9|
|  9|2022|1.050803322E9|
|  5|2019|1.046575362E9|
| 11|2021|1.017852704E9|
|  8|2022| 8.24018941E8|
| 11|2022| 8.21771702E8|
|  6|2018| 8.05480431E8|
| 12|2018| 7.68446952E8|
| 10|2021| 7.53034281E8|
|  8|2020| 7.34045971E8|
|  8|2021| 7.33022625E8|
| 10|2022|   7.299149E8|
| 12|2020| 7.01857916E8|
| 12|2021| 6.96631601E8|
|  7|2021|  6.9078686E8|
|  3|2021| 6.88715085E8|
|  4|2022| 6.86140606E8|
+---+----+-------------+
only showing top 20 rows



In [ ]:
rank_mes_menor_gastos.show()

+---+----+-----------+
|mes| ano|valor_total|
+---+----+-----------+
|  1|2011|        0.0|
|  1|2012|    63438.0|
|  1|2015|    70762.0|
|  3|2011|  5916602.0|
|  1|2014|  7200892.0|
|  1|2013|  7365170.0|
|  2|2011|  9015824.0|
|  4|2011|2.2368382E7|
|  9|2012|2.2551506E7|
| 10|2011|2.2824025E7|
|  9|2013|2.4033908E7|
|  6|2013|2.4296229E7|
|  2|2015|2.6648963E7|
|  6|2012|2.8415455E7|
|  9|2011|2.8885388E7|
|  2|2014|2.9264709E7|
|  8|2015|2.9680735E7|
|  2|2012| 3.018917E7|
| 10|2014|3.0708503E7|
|  7|2012|3.1313881E7|
+---+----+-----------+
only showing top 20 rows



In [ ]:
media_mensal_anual.show()

+----+--------------------+
| ano|        media_mensal|
+----+--------------------+
|2022|   8943865.834070796|
|2021|1.0236525438515082E7|
|2020|   7564575.443089431|
|2019|  6353167.4020715635|
|2018|   5347474.279965004|
|2017|   4662496.313159747|
|2016|  2473558.3476635516|
|2015|   627312.6907467532|
|2014|   655585.9293023256|
|2013|   488017.9815016322|
|2012|  459442.17279046675|
|2011|   526528.1503759399|
+----+--------------------+



In [29]:
rank_ano_maior_gastos.show()

+----+-------------+
| ano|  valor_total|
+----+-------------+
|2021|8.823884928E9|
|2022|8.085254714E9|
|2019|6.747063781E9|
|2017|6.625407261E9|
|2018|6.112163102E9|
|2020|5.582656677E9|
|2016|2.646707432E9|
|2015| 7.72849235E8|
|2014| 7.04754874E8|
|2012| 4.62658268E8|
|2013| 4.48488525E8|
|2011|  3.5014122E8|
+----+-------------+



In [30]:
rank_ano_menor_gastos.show()

+----+-------------+
| ano|  valor_total|
+----+-------------+
|2011|  3.5014122E8|
|2013| 4.48488525E8|
|2012| 4.62658268E8|
|2014| 7.04754874E8|
|2015| 7.72849235E8|
|2016|2.646707432E9|
|2020|5.582656677E9|
|2018|6.112163102E9|
|2017|6.625407261E9|
|2019|6.747063781E9|
|2022|8.085254714E9|
|2021|8.823884928E9|
+----+-------------+



In [31]:
rank_ano_maior_gastos.repartition(1).write.csv("/tmp/output-rank_ano_maior_gastos")

In [34]:
rank_ano_menor_gastos.repartition(1).write.csv("/tmp/output-rank_ano_menor_gastos")

In [ ]:
rank_mes_maior_gastos.repartition(1).write.csv("/tmp/output-rank_mes_maior_gastos")

In [ ]:
rank_mes_menor_gastos.repartition(1).write.csv("/tmp/output-rank_mes_menor_gastos")

In [ ]:
media_mensal_anual.repartition(1).write.csv("/tmp/output-media_mensal_anual")